In [7]:
from src.ppo import Agent, evaluate
import torch 
import gymnasium as gym
from gymnasium.wrappers.vector import NumpyToTorch

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [2]:
def create_env(render_mode: str = 'human'):
    env = gym.make_vec('LunarLander-v3', render_mode = render_mode)
    env = NumpyToTorch(env)

    return env 

In [3]:
agent = Agent.load_agent('models/lunarlander_agent.pt', device)
agent.actor.eval()
agent.critic.eval()
env = create_env()

evaluate(env, 3, agent)

evaluation reward: 267.6382861012411
evaluation reward: 242.05109553856823
evaluation reward: 277.16349176177346
average reward: 262.2842911338609


In [4]:
env.close()

In [5]:
env = create_env('rgb_array')
evaluate(env, 100, agent)

evaluation reward: 262.5747043291117
evaluation reward: 261.8672607022617
evaluation reward: 286.1018743811111
evaluation reward: 261.77590037724144
evaluation reward: 263.1245100811454
evaluation reward: 298.9572241867387
evaluation reward: 304.1029325650778
evaluation reward: 276.8158851284322
evaluation reward: 249.89366123394282
evaluation reward: 291.33149775524885
evaluation reward: 269.5535478974354
evaluation reward: 272.7279249743727
evaluation reward: 280.02131150318826
evaluation reward: 274.11574971228686
evaluation reward: 306.96222514896914
evaluation reward: 302.5195611927991
evaluation reward: 261.3171457897505
evaluation reward: 303.08448717305123
evaluation reward: 275.8715725907149
evaluation reward: 283.3515855310328
evaluation reward: 274.4596502083976
evaluation reward: 259.87865216204057
evaluation reward: 235.62617463412695
evaluation reward: 279.7345503793015
evaluation reward: 294.49291565285614
evaluation reward: 280.1611539813739
evaluation reward: 287.10061

In [6]:
env.close()